# Undoing Things

## 1. Undoing Changes Before Committing

Being able to revert our changes is one of the most powerful features offered by version control systems. There's a bunch of different techniques available depending on which changes we need to undo. In this video and the next few coming up, we'll talk about the most common ways to revert changes in Git and when to use each approach. 

### 1. Undoing Files Not In Staging Area

For example, you might find yourself in a situation where you've made a bunch of changes to a file but decide that you don't want to keep them. You can change a file back to its earlier committed state by using the `git checkout` command followed by the name of the file you want to revert. Speaking of, let's try this out using our scripts repository. We'll edit our all checks pi script and remove the check reboot function, then save and go back to the command line.

```python
import os, sys

def main():
    if check_reboot():
        print('Pending reboot')
        sys.exit(1)
    print('Everything ok')
    sys.exit(0)
main()
```


Cool. We've made our change. Let's try our script and see what happens.


```
$ python all_checks.py
Traceback (most recent call last):
  File "all_checks.py", line 9, in <module>
    main()
  File "all_checks.py", line 4, in main
    if check_reboot():
NameError: name 'check_reboot' is not defined
```

```
$ git status
On branch 3-Git-and-Github
Your branch is ahead of 'origin/3-Git-and-Github' by 10 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
        modified:   all_checks.py
```


By deleting that function, we've actually broke the script. Let's see what git status has to say about this. As expected, we see that our file is modified and the changes aren't staged yet. Check out how git gives us a couple helpful tips on what to do now. We can run git add to stage our changes or we can run git checkout to discard them. 

If you need help remembering what this command does, think of it this way, you're checking out the original file from the latest storage snapshot. Let's do that now. We'll check out at the original file and then take a look at what git status has to say about it and finally retry our script.

```
$ git checkout all_checks.py
Updated 1 path from the index

$ git status
On branch 3-Git-and-Github
Your branch is ahead of 'origin/3-Git-and-Github' by 10 commits.
  (use "git push" to publish your local commits)
Nothing to commit
```

Done and done. With that, we've demonstrated how we can use git checkout to revert changes to modify files before they get staged. This command will restore the file to the latest storage snapshot, which can be either committed or staged. 

So if you've made additional changes to a file after you've staged it, you can restore the file to the earlier stage version. If you need to check out individual changes instead of the whole file, you can do that using the `-p` flag. This will ask you change by change if you want to go back to the previous snapshot or not. 

### 1.2 Undoing Files Already in Staging Area

That's it for undoing unstaged changes. What if you added the changes to the staging area already? Don't stress. If we realize we've added something to the staging area that we didn't actually want to commit, we can unstage our changes by using the git reset command. 

Staging changes that we don't actually intend to commit happens all the time. Especially if we use a command like git add star, where the star is a file glob pattern used in Bash that expands to all files. This command will end up adding any change done in the working tree to the staging area. While sometimes that might be what we want, it can also lead to some surprises. Let's try it out with an example. First, we'll pretend we're trying to debug a problem in our script. For that, we create a temporary file with the output of our script. Then, we'll add all unstaged changes in our working tree using git add star. Finally, check the status using git status. We can see that this output file, which was supposed to be a temporary file for debugging, has now been staged in our repo but we didn't want to commit it. 

```
$ python all_checks.py > output.txt

$ cat output.txt
Everything ok

$ git add *

$ git status
On branch 3-Git-and-Github
Your branch is ahead of 'origin/3-Git-and-Github' by 10 commits.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
        new file:   output.txt
```

Conveniently, the git status command tells us how to unstage the file right there in the output. The example output mentions the head alias. Remember what that means? That's right. It's the current checked out snapshot. So by running the suggested command, we're resetting our changes to whatever's in the current snapshot. Let's try it out.

```
$ git restore --staged output.txt

$ git status
On branch 3-Git-and-Github
Your branch is ahead of 'origin/3-Git-and-Github' by 10 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
        ../../../Week-1/3-Using-Git/checks/
        ../2-Undoing-Things.ipynb
        output.txt
```


The file is once again untracked in our working tree and no longer staged. You can think of reset as the counterpart to add. With add, you can well add changes to the staging area. With reset, you remove changes from the staging area. You can use git reset dash p to get git to ask you which specific changes you want to reset. Get it. But wait, let's remember to commit our typo fix.

## 2. Amending Commits (`--amend`)

In general, we try to make sure our commits include all the right changes and descriptions. But we're all human and we make mistakes. It's not uncommon for developers and IT specialists to realize that there is an error in a recent commit, which is why it's important to know how to take action and fix it. 

Let's say you just finished committing your latest batch of work, but you've forgotten to add a file that belongs to the same change. You'll want to update the commit to include that change. Or maybe the files were correct, but you realize that your commit message just wasn't descriptive enough. So you want to fix the description to add a link to the bug that you're solving with that commit. What can you do? 

We can solve problems like these using the `--amend` option of the git commit command. When we run git commit --amend, git will take whatever is currently in our staging area and run the git commit workflow to overwrite the previous commit. Let's see this in an example. We'll go to our scripts directory and create two new files using the touch command. Then list the contents of the directory using ls at our Python script and commit it saying that we've added two files.

```
$ ls -l
total 2
-rw-r--r-- 1 BRIAN 197121 295 Aug 14 12:24 all_checks.py
-rw-r--r-- 1 BRIAN 197121   0 Aug 14 12:40 auto-update.py
-rw-r--r-- 1 BRIAN 197121   0 Aug 14 12:40 gather-information.sh
-rw-r--r-- 1 BRIAN 197121  15 Aug 14 12:27 output.txt

$ git add auto-update.py

$ git commit -m'Add two new scripts'
[3-Git-and-Github fb91083] Add two new scripts
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 3-Git-and-Github/Week-2/2-Undoing-Things/scripts/auto-update.py
```

As you can see, the message printed by git says that only one file was added. Our commit message said that we added two files, but we forgot to add one of them. Ouch. Don't panic. We can fix it. We'll start by adding the missing file and then amending our commit.

```
$ git add gather-information.sh

$ git commit --amend
[3-Git-and-Github 93ddfbc] Add two new scripts
 Date: Fri Aug 14 12:41:34 2020 -0700
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 3-Git-and-Github/Week-2/2-Undoing-Things/scripts/auto-update.py
 create mode 100644 3-Git-and-Github/Week-2/2-Undoing-Things/scripts/gather-information.sh
```
We call git commit --amend and an editor opened up showing the commit message and the stats about the commit that we're working with. The list of added files for this commit now includes both files that we wanted to add. Yay. Now that the files have been added, we can also improve our initial commit message which was a bit too short. We'll keep the existing description as the first sentence of our commit, and then add a line of description about the intended purpose of each file.
```
Add two new scripts

# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
#
# Date:      Fri Aug 14 12:41:34 2020 -0700
#
# On branch 3-Git-and-Github
# Your branch is ahead of 'origin/3-Git-and-Github' by 12 commits.
#   (use "git push" to publish your local commits)
#
# Changes to be committed:
#       new file:   auto-update.py
#       new file:   gather-information.sh
#
# Changes not staged for commit:
#       modified:   ../2-Undoing-Things.ipynb
#
# Untracked files:
#       ../../../Week-1/3-Using-Git/checks/
#       output.txt
#
```

We've amended our previous commit to include both files and a better message. You could also just update the message of the previous commit by running the git commit --amend command with no changes in the staging area. 

An important heads up. While git --amend is okay for fixing up local commits, you shouldn't use it on public commits. Meaning, those that have been pushed to a public or shared repository. This is because using --amend rewrites the git history removing the previous commit and replacing it with the amended one. This can lead to some confusing situations when working with other people and should definitely be avoided. So remember, fixing up a local commit with amend is great and you can push it to a shared repository after you fixed it. **But you should avoid amending commits that have already been made public.** If this sounds confusing now, don't worry. We'll mention it again when we talk about collaborating with others through shared repositories. 

We've covered how to fix staged and unstaged changes, and how to fix a commit that was incomplete. Up next, we'll talk about what to do if you come across a bad commit that needs to be completely reverted.

## 3. Rollbacks

Fixing your work before you commit is good. But what happens if it's already been snapshotted by Git? Let's say you host to Git repository on a company server that contains all kinds of useful automation scripts that you and your coworkers use. One morning before coffee, you make a few changes to one of these scripts and commit the updated files. A few hours later, you start to receive tickets from users indicating some part of the script is broken. From the errors they describe, it sounds like the problem is related to your recent changes. Oh, you could look at the code you updated to see if you can spot the bug. But more tickets are pouring in and you want to fix the problem as fast as possible. 

You decided it's time for a rollback. There are a few ways to rollback commits in Git. For now, we'll focus on using the `git revert` command. Git revert doesn't just mean undo. Instead, it creates a commit that contains the inverse of all the changes made in the bad commit in order to cancel them out. 

For example, if a particular line was added in the bad commit, then in the reverted commit, the same line will be deleted. This way you get the effect of having undone the changes, but the history of the commits in the project remains consistent leaving a record of exactly what happened. 

So git revert will create a new commit, that is the opposite of everything in the given commit. We can revert the latest commit by using the **HEAD** alias that we mentioned before. Since we can think of head as a pointer to the snapshot of your current commit, when we pass head to the revert command we tell Git to rewind that current commit, makes sense? To check this out, we'll first add a faulty commit to our example repo.

```python
import os, sys

def check_reboot():
    '''Returns true if the computer has a pending reboot'''
    return os.path.exists('/run/reboot-required')

def main():
    if check_reboot():
        print('Pending reboot')
        sys.exit(1)
    if disk_full():
        print('Disk full')
        sys.exit(1)
    print('Everything ok')
    sys.exit(0)
main()
```

Oops, we use the function that we forgot to define. Okay. It's rollback time. Let's get rid of this faulty code by typing git revert head.

```
Revert "Add call to disk_full function"

Reason: disk_full function not defined

This reverts commit 504737b2ee649644af9fb5f22619dbaa8669db8b.

# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
#
# On branch 3-Git-and-Github
# Your branch is ahead of 'origin/3-Git-and-Github' by 13 commits.
#   (use "git push" to publish your local commits)
#
# Changes to be committed:
#       modified:   3-Git-and-Github/Week-2/2-Undoing-Things/2-Undoing-Things.ipynb
#       modified:   3-Git-and-Github/Week-2/2-Undoing-Things/scripts/all_checks.py
#
# Untracked files:
#       3-Git-and-Github/Week-1/3-Using-Git/checks/
#       3-Git-and-Github/Week-2/2-Undoing-Things/scripts/output.txt
```

So once we issue that git revert command, we're presented with the text editor commit interface that we've all seen before. In this case, we can see that git has automatically added some text to the command indicating it's a rollback. The first-line mentions that it's reverting the commit we just did called Add call to disk full function. The extra description even includes the identifier of the commit that got reverted. While we could use this description as is, it's usually a good idea to add an explanation of why we're doing the rollback. Remember that the goal of these descriptions is to help our future selves understand why things happen. In this case, we'll explain that the reason for the rollback is that the code was calling a function that wasn't defined.

```
$ git revert HEAD
[3-Git-and-Github 2228a24] Revert "Add call to disk_full function"
 2 files changed, 91 deletions(-)
```

Once we're done entering the description, we can exit and save as usual. You'll notice the output that we get from the git revert command looks like the output of the git commit command. This is because git revert creates a commit for us. Since a revert is a normal commit, we can see both the commit and the reverted commit in the log. Let's look at the last two entries in the log using dash P and dash two as parameters. As demonstrated before, the dash P parameter lets us see the patch created by the commit while the dash two perimeter limits the output to the last two entries. 


So in this log, we can see that when we called revert, git created a new commit that's the inverse of the previous one. This removes the lines that we added in the previous commit. We can see that the original commit shows the lines we added by preceding them with a plus sign. The same line shows up with a minus sign in the newer commit message indicating that they were removed. Just like that, the bad commit is reverted and the error stopped. In this example, we reverted the latest commit in our tree. But what if we had to revert a commit that was done before that? Rev up your time machines because in the next video, we're turning back the clock big-time.

## 4. Identifying a Commit

So far we've used the HEAD alias to specify the most recently checked out commit in our Git history. In our bad snapshot example, the error also happened to be in the most recently created commit, but errors can sometimes take a while to be detected. And so, we might need to revert other commits farther back in time. We can target a specific commit by using its commit ID. We've seen commit IDs a few times already. They show up when we're running the git log command, and we also saw the commit ID of the reverted commit in our last example. Commit IDs are those complicated looking strings that appear after the word commit in the log messages. Let's have a look at the latest log entry in our checks repo.

```
$ git log -1
commit 45a09fc77b30896e0799114a068c93a072f15161 (HEAD -> 3-Git-and-Github)
Author: 
Date:   Sun Aug 16 10:47:08 2020 -0700

    G&G week 2: "rollbacks"
```

The commit ID is the 40 character long string after the word commit, you really can't miss it. This long jumble of letters and numbers is actually something called a **hash**, which is calculated using an algorithm called **SHA1**. 

Essentially, what this algorithm does is take a bunch of data as input and produce a 40 character string from the data as the output. In the case of Git, the input is all information related to the commit, and the 40 character string is the commit ID.

Cryptographic algorithms like SHA1 can be really complex, so we won't go too deep into what this means. If you're interested, you'll find links to more information in the next reading. 

### 4.1 Why Use SHA1 Algorithm?

Still you might be wondering, why on earth would you use a long jumble of letters as an ID for commit, instead of incrementing an integer, like 123, etc? To answer that, let's take a quick look at the reason why Git uses a hash instead of a counter, and how that hash is computed. Although SHA1 is a part of the class of cryptographic hash functions, Git doesn't really use these hashes for security. Instead, they're used to guarantee the consistency of our repository.

Having consistent data means that we get exactly what we expect. To quote Git's creator, Linus Torvalds, you can verify the data you get back out is the exact same data you put in. This is really useful in distributed systems like Git because everyone has their own repository and is transmitting their own pieces of data. Computing the hash keeps data consistent because it's calculated from all the information that makes up a commit. The commit message, date, author, and the snapshot taken of the working tree. The chance of two different commits producing the same hash, commonly referred to as a collision, is extremely small. So small, it wouldn't happen by chance. It'd take a lot of processing power to cause this to happen on purpose. If you use a hash to guarantee consistency, you can't change anything in the Git commit without the SHA1 hash changing too.

Remember our discussion about fixing commits with the dash dash amend command? Each time we amend a commit, the commit ID will change. This is why it's important not to use dash dash amend on commits that have been made public.

The data integrity offered by the commit ID means that if a bad disk or network link corrupt some data in your repository, or worse, if someone intentionally corrupt some data, Git can use the hash to spot that corruption. Aha, it will say, the data you've got isn't the data you expected, something went wrong.

### 4.2 Using Commit IDs

Okay, enough backstory. How can you use commit IDs to specify a particular commit to work with, like during a rollback? Let's look at the last two entries in our repo using the git log -2 command.

```
$ git log -2
commit 45a09fc77b30896e0799114a068c93a072f15161 (HEAD -> 3-Git-and-Github)
Author: 
Date:   Sun Aug 16 10:47:08 2020 -0700

    G&G week 2: "rollbacks"

commit 2228a24724d038caca876af328e234ed09a5cbdb
Author: 
Date:   Sun Aug 16 10:43:10 2020 -0700

    Revert "Add call to disk_full function"

    Reason for rollback: the disk_full function is undefined

    This reverts commit 504737b2ee649644af9fb5f22619dbaa8669db8b.
```

Say we realized that we actually liked the previous name of our script, and so we want to revert this commit where we renamed it. First, let's look at that specific commit using git show, which we mentioned in an earlier video.

```
$ git show 2228a24724d038caca876af328e234ed09a5cbdb

diff --git a/3-Git-and-Github/Week-2/2-Undoing-Things/scripts/all_checks.py b/3-Git-and-Github/Week-2/2-Undoing-Things/scripts/all_checks.py
index 392d0ec..07ebb8d 100644
--- a/3-Git-and-Github/Week-2/2-Undoing-Things/scripts/all_checks.py
+++ b/3-Git-and-Github/Week-2/2-Undoing-Things/scripts/all_checks.py
@@ -8,9 +8,6 @@ def main():
     if check_reboot():
         print('Pending reboot')
         sys.exit(1)
-    if disk_full():
-        print('Disk full')
-        sys.exit(1)
     print('Everything ok')
     sys.exit(0)
 main()
\ No newline at end of file
```

We've copied and pasted the commit ID that we wanted to display, and that works. Alternatively, we could provide just the first few characters identifying the commit to the command, and Git will be smart enough to guess which commit ID starts with those characters, as long as there's only one matching possibility. Let's try this out.

```
$ git show 2228
commit 2228a24724d038caca876af328e234ed09a5cbdb
Author: Brian Nguyen <brian.edison.nguyen@gmail.com>
Date:   Sun Aug 16 10:43:10 2020 -0700

    Revert "Add call to disk_full function"

    Reason for rollback: the disk_full function is undefined

    This reverts commit 504737b2ee649644af9fb5f22619dbaa8669db8b.
```

Okay, now that we've seen how we can identify the commit that we want to revert, let's call the git revert command with this identifier. As usual, this will open an editor where we should add a reason for the rollback. In this case, we'll say that the previous name was actually better. Hooray for flip-flopping.

As we called out before, when we generate the rollback, Git automatically includes the ID of the commit that we're reverting. This is useful when looking at a repo with a complicated history that includes a lot of commits. Now, once we save and exit the commit message, Git will actually perform the rollback and generate a new commit with its own ID. See how before the name of our commit the revert command already shows the first eight characters of the commit ID? Let's use git show to look at it.

All right, we've managed to revert a commit that wasn't the most recent one. Well done, time travelers. Over the past several videos we've covered a bunch of ways to undo things in Git. Whether for unstaged changes, staged changes, amending commits, or rolling back changes. If anything still seems unclear, now's a great time to practice these commands on your local computer, try things out, and come up with more examples of use cases you want to test. Up next, we've spun up a handy cheat sheet summarizing all the content we just covered. When you're ready, move on to the next quiz and put all this new knowledge into practice. We've been learning a lot of complex things over the past few videos, so once you're done with that quiz you should reward yourself for getting through this technical detail with a break. You earned it. Grab a coffee, tea, or snack and let the concepts settle for a bit. I'll meet you over the next video.